# Trial 2
something different, maybe. Hope this works! :')

In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

In [2]:
with open("../df.pkl", "rb") as f:
    df = pickle.load(f)

## Tools testing

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from yake import KeywordExtractor
from keybert import KeyBERT
from transformers import pipeline

/root/storion_aggregation_analysis/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### YAKE (Looks good, but too many keywords)

In [4]:
kw_extractor = KeywordExtractor()

In [5]:
text = df.loc[df["event_id"] == "case_1"].iloc[0]["content"]

In [6]:
keywords = kw_extractor.extract_keywords(text)

In [7]:
keywords

[('allowing Elon Musk', np.float64(0.0012443619293702544)),
 ('Elon Musk America', np.float64(0.0016008832896744504)),
 ('Musk America PAC', np.float64(0.005631785765392376)),
 ('Elon Musk', np.float64(0.006004674381945689)),
 ('allowing Elon', np.float64(0.015793797497147603)),
 ('Musk America', np.float64(0.020240416352859676)),
 ('Pennsylvania judge', np.float64(0.02226199325158571)),
 ('Judge Angelo Foglietta', np.float64(0.03145404645119408)),
 ('America PAC', np.float64(0.035297120716982114)),
 ('Pleas Court Judge', np.float64(0.05132581560866452)),
 ('Court Judge Angelo', np.float64(0.05132581560866452)),
 ('Common Pleas Court', np.float64(0.05474920997337077)),
 ('contest through Nov.', np.float64(0.05721328937878896)),
 ('ruling on Monday.', np.float64(0.05721328937878896)),
 ('America PAC attorney', np.float64(0.06127688564125262)),
 ('Nov.', np.float64(0.06536876963257338)),
 ('Monday.', np.float64(0.06536876963257338)),
 ('Pennsylvania', np.float64(0.0772582259727435)),
 ('

In [8]:
kw_sum = 0
kw_sum = sum([keyword[1] for keyword in keywords])
kw_sum

np.float64(0.8351456024090593)

### Keybert (Useless)

In [9]:
model = KeyBERT(model="all-mpnet-base-v2")

In [10]:
keywords_kb = model.extract_keywords(text)
keywords_kb

[('contest', 0.3704),
 ('pac', 0.3642),
 ('election', 0.363),
 ('trump', 0.3499),
 ('musk', 0.3343)]

### NER_xlm-roberta-large-finetuned-conll03-english

In [11]:
fair_ner_pipe = pipeline("token-classification", model="FacebookAI/xlm-roberta-large-finetuned-conll03-english")

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [12]:
raw_entities = fair_ner_pipe(text)

In [13]:
text

'A Pennsylvania judge is allowing Elon Musk\'s America PAC to continue their $1 million a day contest through Nov. 5, according to a ruling on Monday.. . The ruling by Common Pleas Court Judge Angelo Foglietta comes after it was revealed during court proceedings that the contest\'s winners are not randomly selected.. . "The $1 million recipients are not chosen by chance," America PAC attorney Chris Gober said in court on Monday. "We know exactly who will be announced as the $1 million recipient today and tomorrow.". . NIKKI HALEY PENS SUPPORTIVE OP-ED IN FAVOR OF TRUMP AHEAD OF ELECTION DAY: \'EASY CALL\'. . CLICK HERE TO GET THE FOX NEWS APP. . The Associated Press contributed to this report.. . This is a breaking news situation. Check back with us for updates.'

In [14]:
raw_entities

[{'entity': 'I-LOC',
  'score': np.float32(0.99997306),
  'index': 2,
  'word': '▁Pennsylvania',
  'start': 2,
  'end': 14},
 {'entity': 'I-ORG',
  'score': np.float32(0.91677386),
  'index': 5,
  'word': '▁',
  'start': 24,
  'end': 25},
 {'entity': 'I-PER',
  'score': np.float32(0.9999901),
  'index': 7,
  'word': '▁El',
  'start': 33,
  'end': 35},
 {'entity': 'I-PER',
  'score': np.float32(0.99999344),
  'index': 8,
  'word': 'on',
  'start': 35,
  'end': 37},
 {'entity': 'I-PER',
  'score': np.float32(0.9999771),
  'index': 9,
  'word': '▁Musk',
  'start': 38,
  'end': 42},
 {'entity': 'I-ORG',
  'score': np.float32(0.9999908),
  'index': 12,
  'word': '▁America',
  'start': 45,
  'end': 52},
 {'entity': 'I-ORG',
  'score': np.float32(0.76671857),
  'index': 13,
  'word': '▁',
  'start': 53,
  'end': 54},
 {'entity': 'I-ORG',
  'score': np.float32(0.99995124),
  'index': 14,
  'word': 'PAC',
  'start': 53,
  'end': 56},
 {'entity': 'I-ORG',
  'score': np.float32(0.9999951),
  'ind

In [15]:
def combine_same_entities(_text, _raw_entities):
    prev_segment = None
    entities = []
    for segment in _raw_entities:
        segment['word'] = segment['word'].replace('▁', ' ')
        original_entity_word = segment['word']
        entity_word = segment['word'].rstrip()
        segment['end'] = segment['end'] - (len(original_entity_word) - len(entity_word))
        if segment['word'].isspace():
            continue
        appended_to_prev_segment = False
        if prev_segment is not None and prev_segment['entity'] == segment['entity']:
            if prev_segment['end'] == segment['start']:
                entities[-1]['word'] += segment['word']
                appended_to_prev_segment = True
            elif _text[prev_segment['end']:segment['start']].isspace():
                entities[-1]['word'] += _text[prev_segment['end']:segment['start']] + segment['word']
                appended_to_prev_segment = True

            if appended_to_prev_segment:
                entities[-1]['end'] = segment['end']
                entities[-1]['score'] = (entities[-1]['score'] + segment['score'])/2

        if not appended_to_prev_segment:
            original_entity_word = entity_word
            entity_word = entity_word.lstrip()
            segment['start'] = segment['start'] + (len(original_entity_word) - len(entity_word))
            entities.append({
                'entity': segment['entity'],
                'word': entity_word,
                'score': segment['score'],
                'start': segment['start'],
                'end': segment['end']
            })
        prev_segment = segment.copy()
    return entities

In [16]:
combine_same_entities(text, raw_entities)

[{'entity': 'I-LOC',
  'word': 'Pennsylvania',
  'score': np.float32(0.99997306),
  'start': 3,
  'end': 14},
 {'entity': 'I-PER',
  'word': 'Elon  Musk',
  'score': np.float32(0.99998444),
  'start': 34,
  'end': 42},
 {'entity': 'I-ORG',
  'word': 'America PAC',
  'score': np.float32(0.99997103),
  'start': 46,
  'end': 56},
 {'entity': 'I-ORG',
  'word': 'Common  Pleas  Court',
  'score': np.float32(0.999995),
  'start': 167,
  'end': 184},
 {'entity': 'I-PER',
  'word': 'Angelo  Foglietta',
  'score': np.float32(0.9999894),
  'start': 192,
  'end': 207},
 {'entity': 'I-ORG',
  'word': 'America PAC',
  'score': np.float32(0.9999592),
  'start': 373,
  'end': 383},
 {'entity': 'I-PER',
  'word': 'Chris  Gober',
  'score': np.float32(0.99999356),
  'start': 394,
  'end': 404},
 {'entity': 'I-PER',
  'word': 'NIKKI  HALEY',
  'score': np.float32(0.9999871),
  'start': 522,
  'end': 532},
 {'entity': 'I-PER',
  'word': 'TRUMP',
  'score': np.float32(0.9999835),
  'start': 568,
  'end': 

## Approach

In [17]:
from scripts.text_filteration import ContentFilterer

In [18]:
content_filter = ContentFilterer()

In [19]:
selected_df = df.iloc[:200]
selected_df = selected_df._append(df.loc[df['event_id'] == 'case_1'])
selected_df = selected_df._append(df.loc[df['event_id'] == 'case_2'])
selected_df = selected_df._append(df.loc[df['event_id'] == 'case_3'])

In [20]:
selected_df.tail()

,title,author,publication_date,source,url,summary,content,tags,images,event_id
id,,,,,,,,,,
b87065f4-5db8-4fde-8371-93fbec783f10,Judge denies Philadelphia DA's request to bloc...,[Abc News],NaT,ABC News,https://abcnews.go.com/Politics/elon-musk-pac-...,NaN,A Philadelphia judge is allowing Elon Musk’s A...,NaN,[https://i.abcnewsfe.com/a/a150a26a-eef0-4ebb-...,case_1
726a7000-dde4-4299-b964-fffbbb41490f,Apple users can soon upgrade to ChatGPT Plus w...,"[Maxwell Zeff, Devin Coldewey, Manish Singh, K...",NaT,TechCrunch,https://techcrunch.com/2024/11/04/apple-users-...,NaN,Apple products are getting an integration with...,"[Apple, ChatGPT Plus, OpenAI, ChatGPT, AI]",[https://techcrunch.com/wp-content/uploads/202...,case_2
141ce42e-29ea-41e1-a670-a9da3fa26e39,Apple will let you upgrade to ChatGPT Plus rig...,"[Jay Peters, A News Editor Who Writes About Te...",NaT,The Verge,https://www.theverge.com/2024/11/4/24288015/ap...,NaN,Apple’s second iOS 18.2 developer beta include...,NaN,[https://www.theverge.com/icons/native-ad-plac...,case_2
eafa01c6-3078-49bf-913f-71697594fe1b,Heidi Klum unveils Halloween costume as ET wit...,"[Stephanie Giang-Paunon Larry Fink, Stephanie ...",NaT,FOX News,https://www.foxnews.com/entertainment/heidi-kl...,NaN,"The queen of Halloween, Heidi Klum, was ""out o...",[#HeidiHalloween],[https://a57.foxnews.com/static.foxnews.com/fo...,case_3
6e353ebb-749d-4520-be04-650305979a44,Heidi Klum Dresses as E.T. for Her Annual Hall...,NaN,NaT,TMZ,https://www.tmz.com/2024/11/01/heidi-klum-et-c...,NaN,Heidi Klum's Halloween bash is always a wild e...,NaN,[https://imagez.tmz.com/image/92/16by9/2024/10...,case_3


In [21]:
cleaned_contents = []

# For cleaning all the contents
# for idx, row in tqdm(df.iterrows(), desc="Filtering contents with CCF...", total=len(df)):
#     cleaned_content = content_filter.filter_text(row['title'], row['content'])
#     cleaned_contents.append(cleaned_content)

# For cleaning of selected sample set
for idx, row in tqdm(selected_df.iterrows(), desc="Filtering contents with CCF...", total=len(selected_df)):
    cleaned_content = content_filter.filter_text(row['title'], row['content'])
    cleaned_contents.append(cleaned_content)

Filtering contents with CCF...: 100%|██████████| 207/207 [01:49<00:00,  1.88it/s]


In [22]:
# Keyword extractor & NER Model
yake_kw = KeywordExtractor()
ner_pipe = pipeline("token-classification", model="FacebookAI/xlm-roberta-large-finetuned-conll03-english")

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [23]:
# Sample of keyword extraction & NER
_sample_text = " ".join(cleaned_contents[19])
print(_sample_text)
sample_keywords = yake_kw.extract_keywords(_sample_text)
print(sample_keywords)
sample_entities = combine_same_entities(_sample_text, ner_pipe(_sample_text))
sample_entities

In brief Leading AI models were again asked who would win the U.S. presidential election between Kamala Harris and Donald Trump  . While early inquiries suggest Trump would win, AI is now leaning towards Harris to win on Tuesday  When we asked ChatGPT in July who would win between Trump and Harris, the chatbot said it was a virtual toss-up between the two, citing RealClearPolling and The Nation  . On the eve of the election, ChatGPT, the leading generative AI platform, refrained from predicting a definitive winner in a head-to-head contest between Donald Trump and Kamala Harris  . ChatGPT was asked to reply in “leetspeak” and Markdown format, as well as who is more likely to win between Trump and Harris under today's conditions  . Using leetspeak and Markdown in prompt engineering tests ChatGPT’s flexibility. ChatGPT was told to analyze and assess reality, with a focus on statistical facts and the reasoning behind the answer  . Finally, ChatGPT was asked to reply based on what it think

[{'entity': 'I-MISC',
  'word': 'AI',
  'score': np.float32(0.70559794),
  'start': 18,
  'end': 19},
 {'entity': 'I-LOC',
  'word': 'U.S',
  'score': np.float32(0.99997735),
  'start': 63,
  'end': 65},
 {'entity': 'I-PER',
  'word': 'Kamala  Harris',
  'score': np.float32(0.99998856),
  'start': 98,
  'end': 110},
 {'entity': 'I-PER',
  'word': 'Donald  Trump',
  'score': np.float32(0.999958),
  'start': 116,
  'end': 127},
 {'entity': 'I-PER',
  'word': 'Trump',
  'score': np.float32(0.99994826),
  'start': 162,
  'end': 166},
 {'entity': 'I-MISC',
  'word': 'AI',
  'score': np.float32(0.5521796),
  'start': 179,
  'end': 180},
 {'entity': 'I-PER',
  'word': 'Harris',
  'score': np.float32(0.99998987),
  'start': 205,
  'end': 210},
 {'entity': 'I-MISC',
  'word': 'ChatGPT',
  'score': np.float32(0.607669),
  'start': 245,
  'end': 251},
 {'entity': 'I-PER',
  'word': 'Trump',
  'score': np.float32(0.9999187),
  'start': 283,
  'end': 287},
 {'entity': 'I-PER',
  'word': 'Harris',
 

In [24]:
def filter_non_entity_keywords(_keywords, _entities):
    entity_list = set([entity['word'] for entity in _entities])
    _keywords = [keyword for keyword in _keywords if keyword[0] not in entity_list]
    return _keywords

In [36]:
from difflib import SequenceMatcher
from collections import defaultdict

# Function to calculate similarity between two strings
def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Function to create a similarity matrix for all words
def create_similarity_matrix(word_list):
    size = len(word_list)
    matrix = [[0] * size for _ in range(size)]

    for i, word1 in enumerate(word_list):
        for j, word2 in enumerate(word_list):
            if i != j:  # Avoid self-comparison
                matrix[i][j] = calculate_similarity(word1, word2)
    return matrix

# Function to group words based on similarity matrix
def group_similar_words(word_list, top_values_threshold=None):
    similarity_matrix = np.array(create_similarity_matrix(word_list))
    # Dynamic thresholding 
    top_n_values = len(similarity_matrix[similarity_matrix>top_values_threshold]) if top_values_threshold is not None else similarity_matrix.shape[0]//1
    top_n_values = 1 if top_n_values==0 else top_n_values
    top_n_values *= -1
    top_n_similarities = np.partition(similarity_matrix.flatten(), top_n_values)[top_n_values:]
    threshold = np.mean(top_n_similarities)
    print(f"Threshold set to {threshold}")
    groups = []
    visited = set()

    for i, word in enumerate(word_list):
        if word not in visited:
            group = set()
            for j, similarity in enumerate(similarity_matrix[i]):
                if similarity >= threshold:
                    group.add(word_list[j])
            group.add(word)  # Include the current word
            groups.append(list(group))
            visited.update(group)  # Mark all words in this group as visited

    return groups

In [40]:
sample_kw_groups = group_similar_words([keyword[0] for keyword in sample_keywords], top_values_threshold=0.3)
sample_kw_groups

Threshold set to 0.4680822105655832


[['Kamala Harris', 'Harris', 'Kamala'],
 ['win between Trump',
  'suggest Trump',
  'early inquiries suggest',
  'inquiries suggest Trump'],
 ['asked ChatGPT',
  'engineering tests ChatGPT',
  'prompt engineering tests',
  'tests ChatGPT'],
 ['win between Trump',
  'Donald Trump',
  'Donald Trump economic',
  'suggest Trump',
  'Trump'],
 ['Markdown', 'Markdown format'],
 ['win'],
 ['Markdown', 'leetspeak and Markdown'],
 ['asked ChatGPT', 'ChatGPT', 'tests ChatGPT'],
 ['Trump and Kamala', 'Trump', 'Kamala']]

'Kamala  Harris'

In [48]:
SequenceMatcher(None, sample_kw_groups[0][0], sample_entities[2]['word']).ratio()

0.9629629629629629

In [49]:
print(sample_kw_groups[0][0])
print(sample_entities[2]['word'])

Kamala Harris
Kamala  Harris


In [ ]:
def link_entity_relevant_keyword_groups(_keywords, _entities):
  

In [27]:
import marimo as mo